In [18]:
import torch
from transformer_lens import HookedTransformer
from transformer_lens import utils

from functools import partial

from sae_lens import SparseAutoencoder
from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes

import plotly.express as px

torch.set_grad_enabled(False)


In [110]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [36]:
# model: HookedTransformer = HookedTransformer.from_pretrained('gpt2-small', device='cpu')
model: HookedTransformer = HookedTransformer.from_pretrained('gpt2-small')


Loaded pretrained model gpt2-small into HookedTransformer


In [37]:
# layer = 1
# prompt_pos = "Yes, I talk about wedding constantly"
# prompt_neg = "I do not talk about wedding constantly"
# prompt_pos = "Love "
# prompt_neg = "Hate"
prompt_pos = "Anger"
prompt_neg = "Calm"

In [38]:
logits, pos_cache = model.run_with_cache(prompt_pos)
# h_p = pos_cache["resid_pre", layer]

logits, neg_cache = model.run_with_cache(prompt_neg)
# h_n = neg_cache["resid_pre", layer]

# print(h_p.shape, h_n.shape)
# steering = h_p - h_n
# steering.shape

In [39]:
# c = 5
def residual_stream_patching_hook(
    resid,
    hook,
    c
):
    h_p = pos_cache[hook.name]
    h_n = neg_cache[hook.name]
    steering = h_p - h_n

    len_steering = steering.shape[1]
    
    # resid shape is (batch, pos, d_model)
    # print(resid.shape, steering.shape)
    resid[:, :len_steering, :] =  resid[:, :len_steering, :] + c * steering

    return resid

In [40]:
text = "I think you're"
n_samples = 7
hook_name = utils.get_act_name("resid_pre", 7)

with model.hooks(fwd_hooks=[(hook_name, partial(residual_stream_patching_hook, c=5))]):
    # model('testing testing asdf;lkjasdf;l k')
    for i in range(n_samples):
        print("============")
        output = model.generate(text, prepend_bos=True, use_past_kv_cache=False, max_new_tokens=20, verbose=False)
        print(output)

I think you're right, but I don't know if it'll lead to a resolution. It's up to the
I think you're seeing the same pattern with the film press over the year. I'm pretty sure that no one showed
I think you're saying that his government did right with almost all 10 million lives saved still there? It was founded in
I think you're missing out on much potential in all of your minor league releases. Starting right away requires the ALL-
I think you're not getting a full description of what actually went down." This was one of two scenes that began playing
I think you're unfortunately right about this; believe it or not, retailers are already putting up barriers for their base retail
I think you're not joking. I filed for divorce two years back and it has now got up to this very day


In [42]:
love_words = ["love", "like", "adore", "enjoy", "appreciate", "cherish", "admire", "care", "fancy", "favor", "prefer"]
hate_words = ["hate", "dislike", "detest", "abhor", "despise", "scorn", "loathe", "despise", "fuck you"]

def compute_metric(positive_words, layer, n_samples, factor):
    count = 0
    hook_name = utils.get_act_name("resid_pre", layer)

    with model.hooks(fwd_hooks=[(hook_name, partial(residual_stream_patching_hook, c=factor))]):
        for i in range(n_samples):
            output = model.generate(text, prepend_bos=True, use_past_kv_cache=False, max_new_tokens=25, verbose=False)
            for word in positive_words:
                if word in output.lower():
                    count += 1
                    break
        
    return count/n_samples

In [25]:
for l in range(model.cfg.n_layers):
    n_samples = 10
    score = compute_metric(hate_words, l, n_samples, factor=10)
    print(f"layer: {l}, socre: {score}")

KeyboardInterrupt: 

In [43]:
layer = 7 # pick a layer you want.

# sparse_autoencoder = SparseAutoencoder.from_pretrained(
#     "gpt2-small-res-jb", f"blocks.{layer}.hook_resid_pre"
# )

In [44]:
hook_point = f"blocks.{layer}.hook_resid_pre"

saes, sparsities = get_gpt2_res_jb_saes(hook_point)

print(saes.keys())
sae = saes[hook_point]

100%|██████████| 1/1 [00:00<00:00,  1.15it/s]

dict_keys(['blocks.7.hook_resid_pre'])


In [45]:
### cache angner and then pass through sae, find anger feature, add anger feature during the forward pass.

hook_name = utils.get_act_name("resid_pre", layer)



In [46]:

logits, cache = model.run_with_cache("Anger")
anger_hidden_state = cache[hook_name][0, -1, :].unsqueeze(0)

feature_acts = sae(anger_hidden_state).feature_acts[0]
top_values, top_ids = torch.topk(feature_acts, 10)
print(top_values)
print(top_ids)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [ ]:
logits, cache = model.run_with_cache("Sad")
anger_hidden_state = cache[hook_name][0, -1, :].unsqueeze(0)

feature_acts = sae(anger_hidden_state).feature_acts[0]
top_values, top_ids = torch.topk(feature_acts, 10)
print(top_values)
print(top_ids)

tensor([39.3675, 16.9168, 15.9074,  5.2267,  5.0246,  3.7320,  3.0697,  3.0380,
         2.8858,  2.3728])
tensor([ 2045, 21248, 20750, 16744, 20385,   667,  4879,  9875, 16910,  8835])


In [ ]:
print((feature_acts != 0).sum())

tensor(32)


In [ ]:
l1_contribution = top_values[0]/feature_acts.sum()
l1_contribution

tensor(0.3290)

In [61]:
target_feature = top_ids[0]

# only top feature
steering = sae.W_dec[top_ids[0]] * top_values[0]   #sae.W_dec[top_ids[:5]].sum(dim=0) # shape: [d_model]

steering = steering.to(model.W_E.device)

# top 5 features
# steering = torch.stack([sae.W_dec[top_ids[i]] * top_values[i] for i in range(5)]).sum(dim=0)

steering.shape

torch.Size([768])

In [64]:
# c = 5
def residual_stream_patching_hook_sae(
    resid,
    hook,
    c,
    pos
):    
    # resid shape is (batch, pos, d_model)
    # print(resid.shape, steering.shape)
    if pos < resid.shape[1]:
        resid[:, pos, :] = resid[:, pos, :] + c * steering

    return resid


text = "I think you're"
n_samples = 7

# with model.hooks(fwd_hooks=[(hook_name, partial(residual_stream_patching_hook_sae, c=1))]):
#     # model('testing testing asdf;lkjasdf;l k')
#     for i in range(n_samples):
#         print("============")
#         output = model.generate(text, prepend_bos=True, use_past_kv_cache=False, max_new_tokens=20, verbose=False)
#         print(output)

In [120]:
def proportion_positive(sentences):
    data = sentiment_pipeline(sentences)
    n_positive = sum([1 for d in data if d['label'] == 'POSITIVE'])
    # print("================")
    # print(sentences)
    # print(data)
    # print("================")
    return n_positive

In [121]:
love_words = ["love", "like", "adore", "enjoy", "appreciate", "cherish", "admire", "care", "fancy", "favor", "prefer"]
# angry_words = ["angry", "mad", "pissed", "irritated", "annoyed", "frustrated", "enraged", "furious", "infuriated", "outraged", "resentful", "agitated", "aggravated", "bitter", "hate", "fuck"]
# angry_words = [
#     'hate', 'despise', 'loathe', 'detest', 'abhor', 'resent', 'dislike', 'disgust', 'anger', 'fury',
#     'rage', 'wrath', 'irritation', 'annoyance', 'frustration', 'bitterness', 'spite', 'vengeance',
#     'revenge', 'resentment', 'hostility', 'animosity', 'contempt', 'scorn', 'disdain', 'malice',
#     'venom', 'rancor', 'enmity', 'aversion', 'repulsion', 'revulsion', 'antipathy', 'ire',
#     'indignation', 'outrage', 'mad', 'furious', 'infuriated', 'incensed', 'irate', 'livid',
#     'seething', 'enraged', 'ballistic', 'pissed', 'aggravated', 'exasperated', 'disgusted',
#     'appalled', 'revolted', 'sickened', 'nauseated', 'fed up', 'sick and tired', 'bitter',
#     'vengeful', 'spiteful', 'vindictive', 'hostile', 'antagonistic', 'contemptuous', 'scornful',
#     'disdainful', 'evil', 'cruel', 'mean', 'nasty', 'vicious', 'vile', 'wicked', 'malicious',
#     'malevolent', 'hateful', 'venomous', 'caustic', 'virulent', 'toxic', 'noxious', 'poisonous',
#     'vitriolic', 'acrimonious'
# ]

batch_size = 64
tokens = model.to_tokens(text, prepend_bos=True)
print(tokens.shape)
batch_tokens = tokens.repeat((batch_size, 1))
print(batch_tokens)

def compute_metric(related_words, n_samples, coef, pos, max_new_tokens=20):
    count = 0

    with model.hooks(fwd_hooks=[(hook_name, partial(residual_stream_patching_hook_sae, c=coef, pos=pos))]):
        for i in range(n_samples):
            output = model.generate(batch_tokens, prepend_bos=True, use_past_kv_cache=False, max_new_tokens=max_new_tokens, verbose=False)
            strings = model.to_string(output)

            count += proportion_positive(strings)

            # for s in strings:
            #     for word in related_words:
            #         if word in s.lower():
            #             count += 1
            #             break
        
    return count/(n_samples * batch_size)


n_samples = 1
max_new_tokens = 20
n_positions = 10
cs = [0, 0.5, 1, 5, 7, 10, 15, 20]
score_matrix = torch.zeros((n_positions, len(cs)))
for pos in range(n_positions):
    for ci, c in enumerate(cs):
        score = compute_metric(angry_words, n_samples, c, pos, max_new_tokens)
        score_matrix[pos, ci] = score

        print(f'pos: {pos}, c: {c}, socre: {score}')

torch.Size([1, 5])
tensor([[50256,    40,   892,   345,   821],
        [50256,    40,   892,   345,   821],
        [50256,    40,   892,   345,   821],
        [50256,    40,   892,   345,   821],
        [50256,    40,   892,   345,   821],
        [50256,    40,   892,   345,   821],
        [50256,    40,   892,   345,   821],
        [50256,    40,   892,   345,   821],
        [50256,    40,   892,   345,   821],
        [50256,    40,   892,   345,   821],
        [50256,    40,   892,   345,   821],
        [50256,    40,   892,   345,   821],
        [50256,    40,   892,   345,   821],
        [50256,    40,   892,   345,   821],
        [50256,    40,   892,   345,   821],
        [50256,    40,   892,   345,   821],
        [50256,    40,   892,   345,   821],
        [50256,    40,   892,   345,   821],
        [50256,    40,   892,   345,   821],
        [50256,    40,   892,   345,   821],
        [50256,    40,   892,   345,   821],
        [50256,    40,   892,   345,

KeyboardInterrupt: 

In [114]:
toks = model.to_str_tokens(text)
x_labels = toks + [f"pos_{i}" for i in range(len(toks), n_positions)]
fig = px.imshow(score_matrix, y=x_labels, x=[str(c) for c in cs], color_continuous_scale="RdBu", color_continuous_midpoint=0)
fig.show()

In [115]:

sum_over_pos = score_matrix.sum(0)
sum_over_pos.shape

px.line(y=sum_over_pos, x=cs, title="Sum of scores over positions", markers=True, labels={'x': "coefficient", "y": "angry score"}).show()

In [116]:
sum_over_pos = score_matrix.sum(0)
sum_over_pos.shape


px.line(y=score_matrix.sum(1), x=x_labels, title="Sum of scores over positions", markers=True, labels={'x': "pos", "y": "angry score"}).show()

In [117]:

model.to_str_tokens(text)

['<|endoftext|>', 'I', ' think', ' you', "'re"]

In [109]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
data = ["I love you", "I hate you"]
sentiment_pipeline(data)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [ ]:
def proportion_positive(sentences):
    data = sentiment_pipeline(sentences)
    n_positive = sum([1 for d in data if d['label'] == 'POSITIVE'])
    return n_positive/len(data)


In [119]:
sentiment_pipeline(["preceding features are connected directly by some weight"])

[{'label': 'NEGATIVE', 'score': 0.5256782174110413}]